# Per-Sample Bottleneck 

This notebook shows how to apply the Per-Sample Bottleneck to pretrained ImageNet models. 

Ensure that `./data/imagenet` points to your copy of the ImageNet dataset. 

In [ ]:
import torch
import torchvision.models 
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize, Normalize
import matplotlib.pyplot as plt 
import os
from tqdm import tqdm_notebook
import json
from PIL import Image
import numpy as np

from IBA.pytorch import IBA, tensor_to_np_img
from IBA.utils import plot_heatmap

## Loading Data and Model

In [ ]:
imagenet_dir = './data/imagenet'

dev = torch.device('cuda:0')

# select a model to analyse
# model = torchvision.models.vgg16(pretrained=True)
model = torchvision.models.resnet50(pretrained=True)
# model = torchvision.models.inception_v3(pretrained=True)
model.to(dev).eval()

# load the data
if type(model) == torchvision.models.inception.Inception3:
    image_size = 299
else:
    image_size = 224
    
valset = ImageFolder(
    os.path.join(imagenet_dir, 'validation'),
    transform=Compose([
        CenterCrop(256), Resize(image_size), ToTensor(), 
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

trainset = ImageFolder(
    os.path.join(imagenet_dir, 'train'),
    transform=Compose([
        CenterCrop(256), Resize(image_size), ToTensor(), 
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]))

with open('imagenet_class_index.json') as f:
    idx2class = {int(k): v[1] for k, v in json.load(f).items()}

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
img, target = valset[0]

### Insert the bottleneck into the model

You can experiment with the location of the bottleneck.

In [ ]:
if type(model) == torchvision.models.vgg.VGG:
    iba = IBA(model.features[17])
elif type(model) == torchvision.models.resnet.ResNet:
    iba = IBA(model.layer2)
elif type(model) == torchvision.models.inception.Inception3:
    iba = IBA(model.Mixed_5b)

## Estimate Mean and Variance

Here, we estimate the mean and variances of the feature map. It is important for measuring the amount of information transmitted.

In [ ]:
iba.estimate(model, trainloader, device=dev, n_samples=10000, progbar=True)

In [ ]:
neuron = (12, 3, 4)
print("Neuron at position {:} has mean {:.2f} and std {:.2f}".format(
    neuron, iba.estimator.mean()[neuron],  iba.estimator.std()[neuron]))

## Display Heatmaps for some random samples



In [ ]:
%matplotlib inline

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(20, 6))
np.random.seed(0)
for ax, sample_idx in zip(axes.flatten(), np.random.choice(50000, 10)):
    img, target = valset[sample_idx]
    img = img[None].to(dev)
    
    # execute the model on a given sample and return the target NLL
    model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target].mean()
    
    # generate the heatmap
    heatmap = iba.heatmap(img, model_loss_closure)
    
    # reverse the data pre-processing for plotting the original image
    np_img = tensor_to_np_img(img[0])
    
    # show the heatmap
    plot_heatmap(heatmap, np_img,  ax=ax)
    ax.set_title(idx2class[target])
    
fig.suptitle("model: {}".format(type(model).__name__))
plt.show()

## Monkey image

In [ ]:
img = np.array(Image.open("./monkeys.jpg"))
img = (img.transpose(2, 0, 1) / 255)
target = 382  # 382: squirrel monkey

# preprocess image
img  = Compose([
    Resize(image_size), ToTensor(),  
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])(Image.open("./monkeys.jpg"))

model_loss_closure = lambda x: -torch.log_softmax(model(x), 1)[:, target].mean()
heatmap = iba.heatmap(img[None].to(dev), model_loss_closure) 
ax = plot_heatmap(heatmap, tensor_to_np_img(img))
_ = ax.set_title(idx2class[target])
plt.show()